# Image segmentation by $k$-means

The objective of this lab is to show how to use $k$-means to segment images. 

You will find below the code for segmenting a greyscale image into $k$ images, each with a specific grey level, by $k$-means clustering. We refer to each such image as a **mask**.

## To do

1. Apply $k$-means to each mask so as to identify different parts / objects of the original image. Test on various images.
2. Adapt the method to color images.


## Import packages

In [ ]:
%pylab inline

In [ ]:
import scipy as sp

## Load images

In [ ]:
# load the raccoon image as a numpy array
try:
    face = -sp.face(gray=True)
except AttributeError:
    from scipy import misc
    face = -misc.face(gray=True)

In [ ]:
plt.imshow(face, cmap='binary')
plt.show()

In [ ]:
img = face[:500,400:900]

In [ ]:
plt.imshow(img,cmap = 'binary')
plt.axis('off')
plt.show()

## Clustering

In [ ]:
class kmeans:
    '''k-means algorithm
    
    Parameters
    ----------
    n_clusters: int, default: 8
        Number of clusters.
    
    n_init : int, default: 10
        Number of instances of k-means, each with different initial centers. 
        The output is that of the best instance (in terms of inertia).
    
    n_iter: int, default: 300
        Number of iterations for each instance of k-means.
        
    algorithm: "random" or "++", default:"++"
        Algorithm for initializing the centers; "++" corresponds to k-means++.
    
    seed: int, default: None
        Seed for the random generation of initial centers.
        
    verbose : boolean, optional
        Verbose mode.
    
    Attributes
    ----------
    clusters_: array, shape(n_samples,)
        Cluster index of each sample.
        
    centers_ : array, shape(n_clusters, n_features)
        Cluster centers.
        
    inertias_: array, shape(n_clusters,)
        Cluster inertias (sum of square distances).
    '''

    def __init__(self, n_clusters=8, n_init=10, n_iter=300, algorithm='++', seed=None, verbose = False):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.n_iter = n_iter
        self.algorithm = algorithm
        self.seed = seed
        self.verbose = verbose
        self.labels_ = None
        self.centers_ = None
        self.inertias_ = None
       
    def fit(self, X):
        '''Cluster data X using k-means
    
        Parameters
        ----------
        X: array, shape(n_samples,n_features)
            Data to cluster.
        '''        
        
        def init_centers(self, X):
            if self.algorithm == 'random':
                # random centers 
                samples = np.random.choice(X.shape[0], size = self.n_clusters)
                centers = X[samples]
            else:
                # k-means++
                centers = np.zeros((self.n_clusters, X.shape[1]))
                sample = np.random.choice(X.shape[0])
                centers[0] = X[sample]
                distance = np.full(X.shape[0], np.inf)
                for j in range(1,self.n_clusters):
                    distance = np.minimum(np.linalg.norm(X - centers[j], axis=1), distance)
                    p = np.square(distance) / np.sum(np.square(distance))
                    sample = np.random.choice(X.shape[0], p = p)
                    centers[j] = X[sample]
            return centers
        
        def compute_centers(self, X, clusters):
            centers = []
            for j in range(self.n_clusters):
                index = np.where(clusters == j)[0]
                if len(index):
                    centers.append(np.mean(X[index],axis = 0))
                else:
                    # reinit center in case of empty cluster
                    centers.append(X[np.random.choice(X.shape[0])])
            return np.array(centers)

        def compute_distances(self, X, centers):
            distances = []
            for j in range(self.n_clusters):
                distances.append(np.linalg.norm(X - centers[j], axis=1))
            return np.array(distances)
            
        def compute_inertias(self, X, centers, clusters):
            inertias = []
            for j in range(self.n_clusters):
                index = np.where(clusters == j)[0]
                inertias.append(np.sum(np.square(np.linalg.norm(X[index] - centers[j], axis=1))))
            return np.array(inertias)
    
        def single_run_kmeans(self, X):
            centers = init_centers(self, X)
            for i in range(self.n_iter):
                centers_old = centers.copy()
                distances = compute_distances(self, X, centers)
                clusters = np.argmin(distances, axis=0)  
                centers = compute_centers(self, X, clusters)
                if np.array_equal(centers, centers_old):
                    break
            inertias = compute_inertias(self, X, centers, clusters)
            return clusters, centers, inertias
            
        np.random.seed(self.seed)
        best_inertia = None
        # select the best instance of k-means
        for i in range(self.n_init):
            if self.verbose:
                print("Instance ",i)
            clusters, centers, inertias = single_run_kmeans(self, X)
            inertia = np.sum(inertias)
            if best_inertia is None or inertia < best_inertia:
                best_clusters = clusters.copy()
                best_centers = centers.copy()
                best_inertias = inertias.copy()
                best_inertia = inertia

        self.clusters_ = best_clusters
        self.centers_ = best_centers
        self.inertias_ = best_inertias
        return self

## Application to greyscale image

In [ ]:
k = 4

In [ ]:
km = kmeans(n_clusters=k, n_init=5, n_iter=100, verbose = True)

In [ ]:
# 1D array
X = img.reshape((-1,1))

In [ ]:
km.fit(X)

In [ ]:
km.centers_

In [ ]:
X_ = km.centers_[km.clusters_]
X_ = np.clip(X_.astype('uint8'), 0, 255)

In [ ]:
img_ = X_.reshape(img.shape)

In [ ]:
plt.imshow(img_,cmap = 'binary')
plt.axis('off')
plt.show()

In [ ]:
def get_masks(img, km):
    masks = []
    for j in range(km.n_clusters):
        X_ = (km.clusters_ == j) * km.centers_[j]
        X_ = np.clip(X_.astype('uint8'), 0, 255)
        masks.append(X_.reshape(img.shape))
    return masks

In [ ]:
masks = get_masks(img, km)

In [ ]:
def show_masks(masks):
    for img in masks:
        plt.figure()
        plt.imshow(img, cmap = 'binary', vmin = 0, vmax = 255)
        plt.axis('off')
        plt.show()

In [ ]:
show_masks(masks)

## Application to color image

In [ ]:
import os.path
from urllib.request import urlretrieve as download

In [ ]:
filename = 'flower.jpg'
if not os.path.isfile(filename):
    download('http://farm3.static.flickr.com/2024/2527645219_a4931e5a26.jpg', filename)

In [ ]:
download('http://farm3.static.flickr.com/2024/2527645219_a4931e5a26.jpg', 'flower.jpg')

In [ ]:
img = imread('./flower.jpg')
# 2D array
X = img.reshape(-1,3)

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.show()